# Model ensamble

The easiest way to pool the predictions of a set of classifiers (to ensemble the classifiers) is to average their predictions at inference time. 

More sofisticated way to ensemble classifiers is to do a weighted average, where the weights are learned on the validation data. Typically, the better classifiers are given a higher weight, and the worse classifiers are given a lower weight. To search for a good set of ensembling weights, we used SLSQP optimization algorithm.

In [1]:
%%capture

import os
import pickle
import numpy as np

import src.model_ensamble as ensamble

from src.data.loaders import load_and_clean_data
from src.definitions import ROOT_PATH
from src.definitions import TEST_PATH

from scipy.optimize import minimize

ROWS = 10000

models = ensamble.init_models()

tokenizer_path = os.path.join(
    ROOT_PATH, "models/{}".format("tokenizer.pkl"))

with open(tokenizer_path, "rb") as file:
    tokenizer = pickle.load(file)

# Load validation reviews
val_samples, val_labels = load_and_clean_data(path=TEST_PATH, nrows=ROWS)
sequences = tokenizer.texts_to_sequences(val_samples)

In [2]:
predictions = ensamble.models_prediction(sequences, val_labels, models)
accuracies = np.array([np.mean(np.round(pred) == val_labels) for pred in predictions])

Evaluating: convnet_keras
10000/10000 [==============================] - 1s 105us/step
Accuracy: 0.9466

Evaluating: convnet_lstm
10000/10000 [==============================] - 4s 398us/step
Accuracy: 0.9506

Evaluating: lstm
10000/10000 [==============================] - 14s 1ms/step
Accuracy: 0.9531



In [3]:
SCALE_FACTOR = -100.0

def objective_function(x):
    ensamble_predictions = ensamble.ensamble_prediction(predictions, weights=x)
    ensamble_accuracy = np.mean(ensamble_predictions == val_labels)
    
    value = SCALE_FACTOR * ensamble_accuracy
    grads = -accuracies
    return value, grads

In [4]:
x0 = np.zeros((len(predictions), 1)) / len(predictions)
bounds = [(0, 1)] * len(predictions)
constraints = [{
    'type': 'eq',
    'fun': lambda x: 1.0 - np.sum(x) 
}]

result = minimize(objective_function, 
                  x0, 
                  jac=True, 
                  method='SLSQP', 
                  bounds=bounds,
                  constraints=constraints,
                  tol=1e-7, 
                  options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -95.6
            Iterations: 2
            Function evaluations: 13
            Gradient evaluations: 2


In [5]:
print(result.x)
print(result.success)
print(result.message)

[0.32983265 0.33383343 0.33633392]
True
Optimization terminated successfully.


In [6]:
test_samples, test_labels = load_and_clean_data(path=TEST_PATH)
sequences = tokenizer.texts_to_sequences(test_samples)
model_predictions = ensamble.models_prediction(sequences, test_labels, models)

Evaluating: convnet_keras
398949/398949 [==============================] - 22s 54us/step
Accuracy: 0.9484

Evaluating: convnet_lstm
398949/398949 [==============================] - 147s 369us/step
Accuracy: 0.9538

Evaluating: lstm
398949/398949 [==============================] - 551s 1ms/step
Accuracy: 0.9555



In [7]:
ensamble_prediction = ensamble.ensamble_prediction(model_predictions)
mean_ensamble_accuracy = np.mean(ensamble_prediction == test_labels)
print("Mean ensamble accuracy: {:.5f}".format(mean_ensamble_accuracy))

ensamble_prediction = ensamble.ensamble_prediction(model_predictions, weights=result.x)
weighted_ensamble_accuracy = np.mean(ensamble_prediction == test_labels)
print("Weighted mean ensamble accuracy: {:.5f}".format(weighted_ensamble_accuracy))

Mean ensamble accuracy: 0.95841
Weighted mean ensamble accuracy: 0.95841
